## Unit 1, Lesson 3.6 Data Cleaning ##

1. Determine the five most common journals and the total articles for each.
2. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal.
3. BONUS: Identify the open access prices paid by subject area.

In [258]:
#preamble
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.ticker as mtick
%matplotlib inline
import re
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

#Read in the data, using iso-8859-1 encoding
wellcome = pd.read_csv('/Users/teresaoneill/Dropbox/Thinkful/Datasets/wellcome.csv', encoding='iso-8859-1')
wellcome_clean = wellcome
wellcome_clean.columns = ['PMID/PMCID', 'Publisher', 'Journal title', 'Article title','Cost']

In [259]:
#First, let's clean the journal titles.

#Use title case for the journal title field.
#Strip trailing whitespace.
#Clean inconsistent journal names by removing 'the' and spelling out abbreviations.
#NOTE: Wouldn't it be better to use a dictionary for 'replace', so it only has to be run once?
#What is the best way to implement a dictionary here?

wellcome_clean['Journal title'] = wellcome_clean['Journal title'].apply(lambda x: str(x).title())
wellcome_clean['Journal title'] = wellcome_clean['Journal title'].apply(lambda x: str(x).strip())

wellcome_clean['Journal title'] = wellcome_clean['Journal title'].apply(lambda x: str(x).replace('J ', 'Journal of '))
wellcome_clean['Journal title'] = wellcome_clean['Journal title'].apply(lambda x: str(x).replace('The Journal', 'Journal'))
wellcome_clean['Journal title'] = wellcome_clean['Journal title'].apply(lambda x: str(x).replace('Org ', 'Organic '))
wellcome_clean['Journal title'] = wellcome_clean['Journal title'].apply(lambda x: str(x).replace('Med ', 'Medicinal '))

print(wellcome_clean['Journal title'].unique())

['Psychological Medicine' 'Biomacromolecules' 'Journal of Medicinal Chem'
 'Journal of Organic Chem' 'Journal Of Medicinal Chemistry'
 'Journal Of Proteome Research' 'Mol Pharm' 'Acs Chemical Biology'
 'Journal Of Chemical Information And Modeling' 'Biochemistry'
 'Gastroenterology' 'Journal Of Biological Chemistry'
 'Journal Of Immunology' 'Acs Chemical Neuroscience' 'Acs Nano'
 'American Chemical Society' 'Analytical Chemistry'
 'Bioconjugate Chemistry' 'Journal Of The American Chemical Society'
 'Chest' 'Journal Of Neurophysiology' 'Journal Of Physiology'
 'American Journal Of Psychiatry' 'Americal Journal Of Psychiatry'
 'Behavioral Neuroscience' 'Emotion' 'Health Psychology'
 'Journal Of Abnormal Psychology'
 'Journal Of Consulting And Clinical Psychology'
 'Journal Of Experimental Psychology:  Animal Behaviour Process'
 'Journal Of Experimental Psychology: Human Perception And Performance'
 'Journal Of Family Psychology' 'Psychological Assessment'
 'Psychological Review'
 'Psycho

In [260]:
#Now let's clean the Price column.
#Use a regex to assign the numerical information, including decimal point, to its own column.
#Use a regex to assign currency symbols to their own column.
#A Caveat: For now, Price does not have any currency converstion, so the values should not really be compared.

wellcome_clean['Price'] = wellcome_clean['Cost'].apply(lambda x: re.findall(r'\d+[.]\d+', x))
wellcome_clean['Currency'] = wellcome_clean['Cost'].apply(lambda x: re.findall(r'$', x))
wellcome_clean['Currency'] = wellcome_clean['Cost'].apply(lambda x: re.findall(r'£', x))


wellcome_clean['Price'] = wellcome_clean['Price'].apply(lambda x: x[0] if x else 0)
wellcome_clean['Price'] = wellcome_clean['Price'].apply(lambda x: pd.to_numeric(x))

In [261]:
#Check Price for anomalous values.
#ax = sns.distplot(wellcome_clean['Price'])
#ax.set(xlabel='', ylabel='', title='Prices')
#plt.show()

#Several articles are priced at 999999.00, probably a missingness indicator.
#Replace this value with the mean article price for that journal. 
#NOTE: That's not working.

wellcome_clean[wellcome_clean == 999999.00] = np.nan
wellcome_clean['Price'] = wellcome_clean['Price'].fillna(wellcome_clean['Price'].groupby(wellcome_clean['Journal title']).mean())

#Recheck distribution.

outliers = wellcome_clean[wellcome_clean['Price'] > wellcome_clean['Price'].quantile(.99)]
print(outliers)

#There appear to be some high outliers with implausible values.
#Let's start with the highest one to find out what's going on.

#The max value is $201024.00.
#Other articles in the same journal are $2010.24, so this appears to be a decimal placement error. 
#Move the decimal over to correct the value, recheck the set of outliers for similar anomalies.
#Correct as necessary.

wellcome_clean['Price'].replace(201024.00, 2010.24, inplace=True)
wellcome_clean['Price'].replace(192645.00, 1926.45, inplace=True)
wellcome_clean['Price'].replace(13200.00, 1320.00, inplace=True)
wellcome_clean['Price'].replace(6000.00, 600.00, inplace=True)

outliers = wellcome_clean[wellcome_clean['Price'] > wellcome_clean['Price'].quantile(.99)]
print(outliers)

#Remaining high values appear to be legitimate, so we will not change them.

                             PMID/PMCID                  Publisher  \
490                          PMC3650559                   Elsevier   
526                                 NaN                   Elsevier   
542                             3605578                   Elsevier   
552                 23541370 PMC3744751                   Elsevier   
648                                 NaN                   Elsevier   
668                   PMCID: PMC3594747                   Elsevier   
728                          PMC3748348                   Elsevier   
729                             2875410                   Elsevier   
786                   PMCID: PMC3694306                   Elsevier   
796                          PMC3525981                   Elsevier   
797                                 NaN                   Elsevier   
798            PMCID:\r    PMC3627205\r                   Elsevier   
799                                 NaN                   Elsevier   
800   PMID: 23041239

In [268]:
print(wellcome_clean['Journal title'].value_counts()[:5])
print(wellcome_clean['Price'].groupby(wellcome_clean['Journal title']).describe())

Plos One                           190
Journal Of Biological Chemistry     60
Neuroimage                          29
Nucleic Acids Research              26
Plos Pathogens                      24
Name: Journal title, dtype: int64
                                                    count         mean  \
Journal title                                                            
Academy Of Nutrition And Dietetics                    1.0  2379.540000   
Acs Chemical Biology                                  5.0  1418.186000   
Acs Chemical Neuroscience                             1.0  1186.800000   
Acs Nano                                              2.0   668.140000   
Acta Crystallographica Section D,  Biological C...    1.0   771.420000   
Acta Crystallographica Section D: Biological Cr...    1.0   773.740000   
Acta Crystallographica Section F: Structural Bi...    2.0   796.635000   
Acta Crystallographica, Section D                     1.0   757.180000   
Acta Crystallography D         